In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, StandardScaler, MinMaxScaler

In [ ]:
dados = pd.read_csv('car_ad.csv', encoding="ISO-8859-1")
dados.head(10)

## Filtrando os dados principais

In [ ]:
dados.isna().sum()

In [ ]:
dados.dropna(axis=0, inplace=True)

In [ ]:
dados.car.value_counts()

In [ ]:
dados = dados.groupby(dados.car).filter(lambda x: len(x) > 20)

In [ ]:
dados.head(10)

In [ ]:
sns.histplot(dados['price'])

In [ ]:
dados = dados[dados['price'] > 0]

In [ ]:
binarizer = LabelBinarizer()
encoder = LabelEncoder()

In [ ]:
dados['registration'] = binarizer.fit_transform(dados['registration'])

In [ ]:
dados.registration.value_counts()

In [ ]:
dados.drive = encoder.fit_transform(dados.drive)

In [ ]:
dados.engType = encoder.fit_transform(dados.engType)

In [ ]:
dados.body = encoder.fit_transform(dados.body)

In [ ]:
dados.head(10)

In [ ]:
dados_numericos = dados.select_dtypes(['int', 'float'])

### Checando a correlação entre a variáveis

In [ ]:
sns.heatmap(dados_numericos.corr(), annot=True)

## Explorando visualmente os dados

In [ ]:
ax = dados.plot(kind='scatter', x='mileage', y='price', figsize=(18, 9), s=15, alpha=0.5)
ax.set_xlabel('Milhas', fontsize=20, color='grey', fontweight='bold')
ax.set_ylabel('Preço', fontsize=20, color='grey', fontweight='bold')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.axis(color='red')
ax.set_title("Relação entre preço e milhas rodadas", size=25, fontweight='bold', color='grey')
ax.axvline(dados['mileage'].mean(), color='red', label='Média de Milhas')
ax.axhline(dados['price'].mean(), color='blue', label='Média de Preço')
ax.axvspan(dados['mileage'].mean() - np.std(dados.mileage), dados['mileage'].mean() + np.std(dados.mileage),
           color='red', label='Desvio Padrão Milhas', alpha=0.1)
ax.axhspan(dados['price'].mean() - np.std(dados.price), dados['price'].mean() + np.std(dados.price),
           color='blue', label='Desvio Padrão Preço', alpha=0.1)

plt.legend(frameon=False)
plt.show()

In [ ]:
ax = dados.plot(kind='scatter', x='year', y='price', figsize=(18, 9), s=15, alpha=0.5)
ax.set_xlabel('Ano', fontsize=20, color='grey', fontweight='bold')
ax.set_ylabel('Preço', fontsize=20, color='grey', fontweight='bold')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.axis(color='red')
ax.set_title("Relação entre preço e ano", size=25, fontweight='bold', color='grey')
ax.axhline(dados['price'].mean(), color='blue', label='Média de Preço')
ax.axhspan(dados['price'].mean() - np.std(dados.price), dados['price'].mean() + np.std(dados.price),
           color='blue', label='Desvio Padrão Preço', alpha=0.1)

plt.legend(frameon=False)
plt.show()

In [ ]:
plt.subplots(figsize=(12,6))
plt.hist(dados['price'], bins=35)
plt.show()

In [ ]:
dados_ml = dados.copy()
dummies_car = pd.get_dummies(dados['car'])
dados_ml.drop(['car','model'], inplace=True, axis=1)
dados_ml = pd.concat([dados_ml, dummies_car], axis=1)

In [ ]:
dados_ml.head(10)

## Modelo de Machine Learning para prever o preço dos veículos com base nas características

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
Y = dados_ml.iloc[:,0]
X = dados_ml.iloc[:,1:]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

## GridSearch optional part

In [ ]:
#parameters = {'bootstrap':('False','True'), 'ccp_alpha':[0,2], 'min_impurity_decrease':[0,2], 'n_estimators':[100,150],
              'oob_score':('False','True')}

In [ ]:
#modelo = RandomForestRegressor()
#gridsearch = GridSearchCV(estimator=modelo, param_grid=parameters, cv=4)
#gridsearch.fit(x_train, y_train)

#melhores_parametros = gridsearch.best_params_
#melhor_predicao = gridsearch.best_score_

In [ ]:
#print(melhores_parametros)
#print(melhor_predicao)

In [ ]:
modelo = RandomForestRegressor(bootstrap =True, ccp_alpha = 0, min_impurity_decrease = 2, n_estimators = 100, oob_score = True)
modelo.fit(x_train, y_train)

In [ ]:
predicao = modelo.predict(x_test)

In [ ]:
print(mean_squared_error(y_test, predicao))
print(r2_score(y_test, predicao))